# Functions

Exemplo de chatbot para banco com Functions

In [12]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("AZURE_OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"

RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

client = AzureOpenAI(
    api_version=os.getenv("AZURE_OPENAI_VERSION",""),
    azure_endpoint=RESOURCE_ENDPOINT,
    api_key = API_KEY,
)


In [13]:
def build_message(role, content):
    return {"role":role, "content":content}

In [14]:
from bankdomain import BankFunctions, Boleto, ContaCorrente

functions = BankFunctions(ContaCorrente(numero="123", saldo=1000, apelido="conta principal"),[
        Boleto(100, "123", "Escola"),
        Boleto(200, "456", "Escola"),
        Boleto(1200, "789", "Escola"),
    ],
    [
        ContaCorrente(numero="987", saldo=0, apelido="André"),
        ContaCorrente(numero="654", saldo=0, apelido="João"),
    ])

tools = []
for function in functions.get_functions():
    tools.append({"type": "function", "function": function})



In [15]:
system_prompt = """"Voce é um assistente bancário que ajuda um cliente a transacionar em usa conta. Seja solicito e educado.", "Este é uma simulação de um assistente bancário, que simula um bot de operações bancárias com linguagem natural.\nSe tiver dúvidas das capacidades, pergunte o que ele pode fazer." Data atual: 2024-09-04."""

query = "Olá"

messages = [ 
                build_message("system", system_prompt),
                build_message("user", query),
    
           ]


response = client.chat.completions.create(
  model="gpt-4o",
  messages = messages,
  temperature=1,
  max_tokens=400,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None,
  tools=tools)

response_message = response.choices[0].message
print(response_message)

messages.append(build_message("assistant", response_message.content))

ChatCompletionMessage(content='Olá! Como posso ajudar você hoje?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [16]:
import json

def todict(obj):
    tool_calls = None
    if (obj.tool_calls is not None):
        tool_calls = []
        for tool_call in obj.tool_calls:
            tool_calls.append({"id": tool_call.id, "function": {"name": tool_call.function.name, "arguments": tool_call.function.arguments}, "type": tool_call.type})
    return {"content": obj.content, "role": obj.role, "tool_calls": tool_calls}

def function_call(functionsObject, messages):
        try:
            print('function_call')
            #for message in chat_context["conversation"]:
            #    print(message["role"])
            #    print(message["content"])
            
            response_message = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                tools=tools,
                tool_choice="auto", 
            ).choices[0].message
            print(response_message)
            if response_message.tool_calls is not None:
                messages.append( # adding assistant response to messages
                    todict(response_message)
                )
                for tool_call in response_message.tool_calls:
                        # Call the function. The JSON response may not always be valid so make sure to handle errors
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    function_response = getattr(functionsObject, function_name)(**function_args)
                    messages.append( # adding function response to messages
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": json.dumps(function_response),
                        }
                    ) 
                return function_call(functionsObject, messages)
            else:
                print("not function")
                messages.append( # adding assistant response to messages
                    {
                        "role": response_message.role,
                        "content": response_message.content,
                    }
                )
                print(messages)
                return 
        except Exception as e:
            print(e)
            return

In [17]:
messages.append(build_message("user", "Qual é o saldo da minha conta?"))

function_call(functions, messages)

messages

function_call
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_79Y4bXaN1Ir5pjVwdwpABwqd', function=Function(arguments='{}', name='saldo'), type='function')])
function_call
ChatCompletionMessage(content='O saldo da sua conta é R$ 1.000,00. Há algo mais que eu possa ajudar?', refusal=None, role='assistant', function_call=None, tool_calls=None)
not function
[{'role': 'system', 'content': '"Voce é um assistente bancário que ajuda um cliente a transacionar em usa conta. Seja solicito e educado.", "Este é uma simulação de um assistente bancário, que simula um bot de operações bancárias com linguagem natural.\nSe tiver dúvidas das capacidades, pergunte o que ele pode fazer." Data atual: 2024-09-04.'}, {'role': 'user', 'content': 'Olá'}, {'role': 'assistant', 'content': 'Olá! Como posso ajudar você hoje?'}, {'role': 'user', 'content': 'Qual é o saldo da minha conta?'}, {'content': None, 'role': 'assistant

[{'role': 'system',
  'content': '"Voce é um assistente bancário que ajuda um cliente a transacionar em usa conta. Seja solicito e educado.", "Este é uma simulação de um assistente bancário, que simula um bot de operações bancárias com linguagem natural.\nSe tiver dúvidas das capacidades, pergunte o que ele pode fazer." Data atual: 2024-09-04.'},
 {'role': 'user', 'content': 'Olá'},
 {'role': 'assistant', 'content': 'Olá! Como posso ajudar você hoje?'},
 {'role': 'user', 'content': 'Qual é o saldo da minha conta?'},
 {'content': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_79Y4bXaN1Ir5pjVwdwpABwqd',
    'function': {'name': 'saldo', 'arguments': '{}'},
    'type': 'function'}]},
 {'tool_call_id': 'call_79Y4bXaN1Ir5pjVwdwpABwqd',
  'role': 'tool',
  'name': 'saldo',
  'content': '1000'},
 {'role': 'assistant',
  'content': 'O saldo da sua conta é R$ 1.000,00. Há algo mais que eu possa ajudar?'}]

In [18]:

messages.append(build_message("user", "Pague todos os boletos pendentes que não estouram o saldo e depois transfira 200 reais para a conta do João."))

function_call(functions, messages)

messages


function_call
ChatCompletionMessage(content='Vou começar verificando os boletos pendentes. Em seguida, farei os pagamentos possíveis e a transferência solicitada. Um momento, por favor.', refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_BG4g80PJo74eJf7zwpMSXqJF', function=Function(arguments='{}', name='boletos_pendentes'), type='function')])
function_call
ChatCompletionMessage(content='Aqui estão os boletos pendentes encontrados:\n\n1. Boleto número 123 - Valor: R$ 100,00\n2. Boleto número 456 - Valor: R$ 200,00\n3. Boleto número 789 - Valor: R$ 1200,00\n\nLevando em consideração que o saldo atual é de R$ 1.000,00, vou prosseguir com o pagamento dos boletos que estão dentro deste limite e em seguida farei a transferência de R$ 200,00 para a conta do João. \n\nVou começar pagando os boletos de R$ 100,00 e R$ 200,00.', refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ZC7tV5hXLGChfp

[{'role': 'system',
  'content': '"Voce é um assistente bancário que ajuda um cliente a transacionar em usa conta. Seja solicito e educado.", "Este é uma simulação de um assistente bancário, que simula um bot de operações bancárias com linguagem natural.\nSe tiver dúvidas das capacidades, pergunte o que ele pode fazer." Data atual: 2024-09-04.'},
 {'role': 'user', 'content': 'Olá'},
 {'role': 'assistant', 'content': 'Olá! Como posso ajudar você hoje?'},
 {'role': 'user', 'content': 'Qual é o saldo da minha conta?'},
 {'content': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_79Y4bXaN1Ir5pjVwdwpABwqd',
    'function': {'name': 'saldo', 'arguments': '{}'},
    'type': 'function'}]},
 {'tool_call_id': 'call_79Y4bXaN1Ir5pjVwdwpABwqd',
  'role': 'tool',
  'name': 'saldo',
  'content': '1000'},
 {'role': 'assistant',
  'content': 'O saldo da sua conta é R$ 1.000,00. Há algo mais que eu possa ajudar?'},
 {'role': 'user',
  'content': 'Pague todos os boletos pendentes que não estou